<h1 align="center">Estructura de Datos y Algoritmos II</h1>
<h1 align="center">Algoritmos Paralelos - Filtros Bloom</h1>
<h1 align="center">2024</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

*** 
|[![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)](mailto:calvar52@eafit.edu.co)||[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/EstructuraDatosAlgoritmos2/blob/main/S09c_AlgoritmosParalelosBloomFilter.ipynb)
|-:|:-|--:|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## ¿Qué es un Filtro Bloom?

Un [filtro Bloom](https://en.wikipedia.org/wiki/Bloom_filter) es una estructura de datos probabilística eficiente en espacio, concebida por Burton Howard Bloom en 1970, que se utiliza para probar si un elemento es miembro de un conjunto.

Un filtro Bloom es una estructura de datos diseñada para verificar de manera rápida y eficiente si un elemento es miembro de un conjunto. Sin embargo, como sugiere el título, es una estructura de datos probabilística, lo que significa que puede decirte con certeza si un elemento no está en el conjunto, pero solo con cierta probabilidad si realmente está en el conjunto. Esta característica conlleva la posibilidad de falsos positivos, pero garantiza que no haya falsos negativos, en otras palabras, una consulta devuelve "posiblemente en un conjunto" o "definitivamente no en un conjunto". También es evidente, según la definición anterior, que esto es útil para el problema ancestral en los Sistemas de Software, "Buscar" o más precisamente, buscar de manera eficiente en grandes conjuntos de datos.

## El Problema de la Búsqueda

Existen diferentes estructuras de datos y algoritmos para buscar datos. Cada uno es adecuado para diferentes situaciones. La forma más básica es verificar cada elemento en un conjunto y compararlo con el elemento que se busca. Este es el método de *"búsqueda lineal"*. La complejidad temporal aquí es $O(n)$. En un sistema distribuido a gran escala donde tienes millones, posiblemente miles de millones, de registros, realizar una búsqueda lineal es una mala idea. Sin embargo, casi siempre puedes optimizar esto si tienes alguna idea sobre el conjunto de datos. Si dijera que el conjunto de datos está ordenado, casi instantáneamente uno optaría por la *búsqueda binaria* y la complejidad temporal se reduce a $O(logN)$.

Para optimizar aún más el tiempo de búsqueda, tenemos estructuras de datos como *tablas hash*, *árboles*, etc. Por lo tanto, basándote en tus datos, puedes elegir la estructura de datos y el algoritmo para optimizar la búsqueda.

Uno de los desafíos con las estructuras de datos discutidas anteriormente es que necesitas tener tus datos (completos o parcialmente) poblados antes de que puedas realizar cualquier búsqueda; por ejemplo, para una tabla hash, necesitarías tanto la clave como el valor.

## Cómo Funcionan los Filtros Bloom

Los filtros Bloom utilizan un arreglo de bits y tratan de mapear la presencia de una entrada particular en ese arreglo verificando si el valor del bit está establecido o no.

- **¿Cómo establecemos el bit?** Pasamos el valor a través de una función hash y establecemos el bit correspondiente al valor hash devuelto.
<p>&nbsp;</p>

- **¿En qué se diferencia de un HashMap?** La principal diferencia con respecto a un HashMap es que, en lugar de utilizar una función hash para mapear la entrada a un bit en el arreglo, pasamos la entrada a través de múltiples funciones hash y establecemos cada bit basado en la salida de las funciones hash independientes.

En los HashMaps, las colisiones pueden llevarnos a inferir si un elemento realmente existe, a menos que comparemos los valores. Sin embargo, en los filtros Bloom, solo almacenamos bits como valores y, por lo tanto, no tenemos forma de saber si el bit establecido en 1 implica que el elemento realmente existe, o si está establecido porque otro elemento tenía el mismo valor hash, lo que llevó a la configuración del bit.

## Implicaciones de Uso

Esta distinción es crucial porque implica que los filtros Bloom son excelentes para situaciones donde es aceptable un grado de incertidumbre pero se requiere rapidez y eficiencia en espacio. Por ejemplo, pueden ser muy útiles en bases de datos distribuidas o en cachés de Internet donde verificar rápidamente la no presencia de un elemento puede evitar búsquedas innecesarias o inserciones duplicadas. En cambio, los HashMaps son preferibles cuando se necesita certeza absoluta sobre la presencia o ausencia de elementos, como en aplicaciones que requieren consistencia de datos y precisión.

## Explicación del Algoritmo con Hash Map y Colisiones

veamos el siguiente ejemplo:

Se desea implementar un sistema que utilice un HashMap para almacenar y verificar la presencia de ciertas cadenas, mostrando cómo las colisiones entre las claves hash pueden afectar la exactitud de las verificaciones. Un posible algoritmo sería el siguiente:

```pseudocode
str1 := "Hello"
str2 := "World"
str3 := "Example"
map[hash(str1)] = 1
map[hash(str2)] = 1

// Assume hash(str1) == hash(str2) i.e Collision
If I check for the existence of str3 in the map, I would get 1, even though World did not exist in the map.

str4 := "NewWord"
However, if I find an entry such that map[hash(str4)] != 1, I can say with certaint
```

El algoritmo presentado ilustra cómo se utiliza una función hash en un mapa (HashMap) para verificar la presencia de cadenas en un conjunto de datos representado por este mapa. Veamos paso a paso cómo funciona y qué implica cada operación:

- ***Paso 1: Asignación de Valores: Primero, se asignan cadenas a variables***

```plaintext
str1 := "Hello"
str2 := "World"
str3 := "Example"
```

- ***Paso 2: Hashing y Almacenamiento:*** Luego, las cadenas `str1` y `str2` se procesan a través de una función hash y se almacenan en un mapa con el valor `1`. Esto implica que estas cadenas están presentes en el conjunto:
```plaintext
map[hash(str1)] = 1
map[hash(str2)] = 1
```

- ***Suposición de Colisión:*** Se supone que las funciones hash de `str1` y `str2` resultan en el mismo valor de hash, lo que significa que hay una colisión. Esto significa que ambos valores, "Hello" y "World", están mapeados al mismo lugar en el mapa:

```plaintext
// Assume hash(str1) == hash(str2) i.e Collision
```

- ***Paso 3: Verificación de Existencia:*** Cuando se verifica la existencia de `str3`, "Example", que no fue añadido previamente al mapa, se utiliza su valor hash para buscar en el mapa:

```plaintext
If I check for the existence of str3 in the map, I would get 1, even though World did not exist in the map.
```
Debido a la colisión o a una coincidencia hash con `str1` o `str2`, se devuelve `1`, indicando erróneamente que `str3` podría estar presente, aunque no lo está. Esto es un falso positivo.

- ***Paso 4: Verificación de No Existencia:*** Finalmente, si se verifica otra cadena, `str4`, y su valor hash no está presente en el mapa o es diferente de `1`, se puede afirmar con certeza que `str4` no está en el conjunto:

```plaintext
str4 := "NewWord"
If I find an entry such that map[hash(str4)] != 1, I can say with certainty
```
Si el valor en `map[hash(str4)]` no es `1`, podemos estar seguros de que `str4` definitivamente no está en el mapa.


Este algoritmo ilustra los riesgos de las colisiones en los HashMaps y cómo pueden llevar a falsos positivos, donde el sistema puede indicar incorrectamente la presencia de un elemento no existente. Los falsos negativos no ocurren con los HashMaps; si un elemento no está presente, el mapa correctamente indica su ausencia, siempre que no haya colisiones afectando el resultado. Esto destaca la importancia de elegir funciones hash eficientes que minimicen las colisiones para mantener la integridad de la búsqueda en estructuras de datos basadas en hash.

## Factores que Influyen en los Filtros Bloom

- **Tamaño del Arreglo de Bits (`m`)**: Al igual que en un hashmap, si el tamaño del arreglo es pequeño, aumentan las posibilidades de colisión. Cuando aumentan las posibilidades de colisión, un bit establecido no puede garantizar si el elemento que se busca existe realmente, ya que el bit podría haber sido establecido como resultado de otro valor. Cuanto mayor sea el tamaño del arreglo, menores serán las posibilidades de colisión. Esto ayuda a reducir los falsos positivos, donde un elemento se identifica incorrectamente como presente.
<p>&nbsp;</p>

- **Número de Funciones Hash (`k`)**: Dado que no podemos aumentar infinitamente el tamaño del arreglo de bits (limitado por la memoria), sabemos que es imposible reducir la colisión solo aumentando el tamaño del arreglo. Por lo tanto, para reducir las posibilidades de colisión, en lugar de usar una única función hash, utilizamos `k` funciones hash independientes, cada una mapeando el mismo elemento a una de las posiciones en el arreglo de bits. Para verificar si un elemento está en el conjunto, se pasa a través de las funciones hash. Si todas las `k` posiciones tienen un 1, el elemento posiblemente está en el conjunto. Si alguna posición tiene un 0, el elemento definitivamente no está en el conjunto.
<p>&nbsp;</p>

- **Número de Elementos en el Conjunto (`n`)**: A medida que aumenta el número de elementos en el conjunto, también aumentan las posibilidades de colisión. Este comportamiento es similar al de un HashMap, donde si intentas insertar 100 elementos en un hashmap de tamaño 75, tus posibilidades de colisión siempre aumentarán. Esto se debe a que más elementos aumentan la probabilidad de que diferentes elementos produzcan los mismos valores hash, llevando a más bits siendo establecidos en las mismas posiciones.

## Implicaciones de los Factores

Estos factores interactúan de maneras que afectan significativamente el rendimiento y la utilidad de los filtros Bloom:

- **Equilibrio entre el tamaño y el número de funciones hash**: Un mayor número de funciones hash puede ayudar a distribuir más uniformemente los elementos dentro del arreglo de bits, reduciendo las colisiones, pero cada función hash adicional también requiere más cálculos y más bits que necesitan ser verificados durante las consultas.
<p>&nbsp;</p>

- **Gestión de la capacidad**: A medida que el conjunto de elementos crece, es crucial reevaluar el tamaño del arreglo de bits y el número de funciones hash para mantener un equilibrio adecuado entre el espacio y el tiempo de consulta, así como la tasa de falsos positivos.

Al diseñar un filtro Bloom para una aplicación específica, es esencial considerar estos factores para optimizar tanto la eficiencia del almacenamiento como la precisión de las búsquedas.

## Probabilidad de Falsos Positivos en Filtros Bloom

Los falsos positivos ocurren cuando los bits de un filtro Bloom están configurados (debido a colisiones) pero el elemento nunca fue insertado en el conjunto. La tasa de falsos positivos es un aspecto crucial, influenciado por el tamaño del arreglo de bits (`m`), el número de funciones hash (`k`) y el número de elementos insertados (`n`). La probabilidad de un falso positivo puede minimizarse optimizando `m` y `k` en función del número esperado de elementos.

Más adelante, profundizaremos en cómo construir un filtro Bloom con los valores correctos de `m` y `k` para manejar un número deseado de elementos, con el fin de desarrollar un filtro Bloom con nuestro nivel deseado de tasa de falsos positivos. Antes de eso, exploremos las ventajas de los filtros Bloom.

## Ventajas de los Filtros Bloom

- **Eficiencia Espacial**: Los filtros Bloom usan un arreglo de bits que es extremadamente espacio-eficiente, especialmente en comparación con estructuras de datos tradicionales como listas o tablas hash que necesitan almacenar los elementos completos.
<p>&nbsp;</p>

- **Rapidez de Consultas**: Verificar si un elemento está en el conjunto puede hacerse rápidamente pasando el elemento a través de las `k` funciones hash y verificando los bits en las posiciones calculadas. Esta operación es muy rápida y su tiempo no depende del tamaño del conjunto.
<p>&nbsp;</p>

- **Facilidad de Escalabilidad**: Aumentar el tamaño del filtro Bloom para manejar más elementos o reducir la tasa de falsos positivos es relativamente simple ajustando el tamaño del arreglo de bits y las funciones hash.
<p>&nbsp;</p>

- **Uso en Aplicaciones Distribuidas**: Los filtros Bloom son ideales para entornos distribuidos donde la sincronización y la minimización del ancho de banda son críticas. Como los filtros Bloom no necesitan transferir el conjunto completo de datos, son útiles para aplicaciones como bases de datos distribuidas y cachés de red.

## Cómo Minimizar los Falsos Positivos

Para minimizar la probabilidad de falsos positivos, es crucial ajustar adecuadamente el tamaño del arreglo de bits (`m`) y el número de funciones hash (`k`). La fórmula para calcular la probabilidad de un falso positivo en un filtro Bloom es aproximadamente:

$$P(fp) \approx \left(1 - e^{-kn/m}\right)^k$$

donde:
- $P(fp)$ es la probabilidad de un falso positivo.
- $k$ es el número de funciones hash.
- $n$ es el número de elementos insertados.
- $m$ es el tamaño del arreglo de bits.

Esta fórmula muestra que aumentar el número de bits por elemento y el número de funciones hash puede reducir significativamente la tasa de falsos positivos, pero esto también implica un mayor uso de recursos computacionales y más espacio de almacenamiento. En la siguiente publicación, discutiremos cómo seleccionar estos parámetros de manera efectiva para equilibrar el rendimiento y la precisión en aplicaciones prácticas.

## Ventajas de los Filtros Bloom

Los filtros Bloom son estructuras de datos únicas que ofrecen múltiples beneficios, especialmente en contextos donde la eficiencia y la escalabilidad son críticas:

- **Eficiencia de Espacio**: Utilizan significativamente menos espacio que otras estructuras de datos como las tablas hash o los árboles para representar la pertenencia a un conjunto. Esto se debe a que un filtro Bloom simplemente almacena un arreglo de bits en lugar de los elementos reales, lo que lo hace extremadamente compacto.
<p>&nbsp;</p>

- **Complejidad Temporal**: Tanto las operaciones de inserción como las de consulta tienen una complejidad temporal constante ($O(k)$), independientemente del número de elementos en el conjunto. Esto se logra mediante el uso de múltiples funciones hash que determinan las posiciones en el arreglo de bits rápidamente.
<p>&nbsp;</p>

- **Eficiencia de Red**: Ideal para situaciones donde transmitir un conjunto grande a través de una red es impráctico, ya que solo necesita enviarse el compacto filtro Bloom. Esto es especialmente útil en aplicaciones distribuidas donde minimizar el tráfico de red y la latencia es crucial.
<p>&nbsp;</p>

- **Escalabilidad**: Favorable para conjuntos de datos grandes donde la sobrecarga de estructuras de datos de pertenencia exacta al conjunto sería prohibitiva. Los filtros Bloom permiten gestionar grandes volúmenes de datos con un mínimo impacto en el rendimiento y la utilización de recursos.

## Limitaciones de los Filtros Bloom

A pesar de sus numerosas ventajas, los filtros Bloom también presentan algunas limitaciones que pueden afectar su aplicabilidad en ciertos escenarios:

- **Probabilidad de Falsos Positivos**: Los filtros Bloom siempre tendrán alguna falsa positividad, lo que significa que no se pueden usar en escenarios donde necesitamos saber con certeza si un elemento existe. Son ideales para confirmar si un elemento no está presente en el conjunto, lo cual es suficiente en muchas aplicaciones, pero puede ser un problema en contextos donde la precisión es crítica.
<p>&nbsp;</p>

- **Falta de Soporte para la Eliminación**: Las implementaciones estándar de los filtros Bloom carecen de soporte para eliminar un elemento del conjunto. Sin embargo, existen variantes como los Filtros Bloom de Conteo que manejan las eliminaciones usando contadores en lugar de bits. Estos filtros utilizan un contador para cada bit, permitiendo incrementar o decrementar según se agreguen o eliminen elementos, ofreciendo así una funcionalidad más flexible pero a expensas de un mayor uso del espacio.

En resumen, los filtros Bloom son herramientas poderosas para aplicaciones que pueden tolerar cierta incertidumbre en la verificación de pertenencia al conjunto, ofreciendo un rendimiento y una eficiencia excepcionales en entornos adecuados. Sin embargo, es crucial evaluar las necesidades específicas de la aplicación para determinar si los beneficios de los filtros Bloom superan sus limitaciones en un contexto dado.

## Aplicaciones Prácticas de los Filtros Bloom

Los filtros Bloom son estructuras de datos versátiles que se aplican en una variedad de escenarios tecnológicos, donde la eficiencia, la velocidad y la economía de espacio son prioritarias. A continuación se presentan algunas de las aplicaciones prácticas más destacadas:

- **Sistemas de Bases de Datos - Verificación Eficiente de Registros:** Antes de realizar operaciones costosas de disco o red, los filtros Bloom permiten consultar de manera eficiente si un registro existe. Esto evita operaciones innecesarias y mejora el rendimiento general del sistema al reducir la carga en los recursos de almacenamiento.
<p>&nbsp;</p>

- **Sistemas de Caché - Prevención de Contaminación de Caché:** Los filtros Bloom ayudan a verificar la presencia de un elemento sin necesidad de consultar la caché misma. Esto evita la contaminación de la caché con datos que no se utilizan frecuentemente, manteniendo la caché limpia y eficiente.
<p>&nbsp;</p>

- **Correctores Ortográficos - Verificación Rápida de Palabras:** Los filtros Bloom son utilizados para comprobar rápidamente si una palabra está en un diccionario. Este método proporciona una respuesta instantánea sobre la existencia de la palabra, optimizando así las funcionalidades de los correctores ortográficos en procesadores de texto y aplicaciones educativas.
<p>&nbsp;</p>

- **Blockchain y Criptografía - Verificación de Transacciones:** En el contexto de blockchain, los filtros Bloom permiten verificar si una transacción específica forma parte de un bloque. Esto es crucial para la validación de transacciones y la integridad del blockchain, facilitando una verificación rápida sin revisar cada transacción en detalle.

## Conclusión

Los filtros Bloom, con su enfoque único para la verificación de pertenencia a conjuntos, se han convertido en una herramienta indispensable en escenarios donde la eficiencia y la velocidad son cruciales. A pesar de sus limitaciones, como los falsos positivos y la falta de soporte para la eliminación en implementaciones básicas, sus ventajas los hacen valiosos en muchas aplicaciones tecnológicas modernas.



### Conclusiones



### El Caso de los Filtros Bloom

Imagina un escenario donde quieres saber si un artículo no existe en un conjunto dado. Un escenario donde es suficiente saber que un artículo ciertamente no está en un conjunto dado o donde el costo de identificar erróneamente un artículo como presente en un conjunto no es crítico en comparación con identificar erróneamente un artículo como ausente. ¿Qué significa esto? Expliquemos con un ejemplo.

Considera el problema de elegir un ID de correo electrónico para un nuevo usuario de Gmail. ¡Hay aproximadamente 1.8 mil millones de usuarios de Gmail! Si tuvieras que verificar en una base de datos tan enorme cada vez que un usuario se registra e intenta elegir un nuevo ID de correo electrónico, ¡el usuario podría rendirse antes de que pudieras buscar a través de los datos! ¿Cuáles son las restricciones aquí?

- Quieres optimizar la velocidad de búsqueda.
- Quieres optimizar el espacio requerido para realizar esta búsqueda.
- Puede ser aceptable decir que un nombre de usuario ya existe, incluso si no es así. El usuario puede elegir uno nuevo. ¡Un ID de correo electrónico duplicado, por otro lado, es un gran problema!

Conociendo todo esto sobre nuestro conjunto de datos, ¿podemos optimizar la búsqueda? Los filtros Bloom nos ayudan exactamente con eso.


### Explicación de los Filtros Bloom

Los filtros Bloom requieren funciones hash y un arreglo de bits. ¡No necesitas los elementos en la estructura de datos para realizar la búsqueda! Continuemos con el ejemplo del caso de uso anterior. Para mantener las cosas simples, usaremos 2 funciones hash y un arreglo de bits de tamaño 10. Consideremos que tenemos los siguientes IDs de correo electrónico para almacenar en un filtro Bloom: "tech@gmail.com" y "wisdom@gmail.com".

Utilicemos dos funciones hash simples en este ejemplo:

**Función Hash 1**: hash1(email) = valor ASCII del primer carácter módulo 10

**Función Hash 2**: hash2(email) = valor ASCII del último carácter módulo 10

Calculemos los valores hash para cada ID de correo electrónico:

Para “tech@gmail.com” (usando los valores ASCII de “t” y “h”):
- hash1(tech) = 116 módulo 10 = 6
- hash2(tech) = 104 módulo 10 = 4

Para “wisdom@gmail.com” (usando los valores ASCII de “w” y “m”):
- hash1(wisdom) = 119 módulo 10 = 9
- hash2(wisdom) = 109 módulo 10 = 9

Ahora, configuraremos los bits correspondientes en el filtro Bloom para cada ID de correo electrónico:

Para "tech@gmail.com": Establecer bits en las posiciones 6 y 4.
Para "wisdom@gmail.com": Establecer bits en la posición 9 (duplicado en la misma posición).

Después de procesar todos los IDs de correo electrónico, el filtro Bloom podría verse así:

| Índice: | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
|---------|---|---|---|---|---|---|---|---|---|---|
| Bit:    | 0 | 0 | 0 | 0 | 1 | 0 | 1 | 0 | 0 | 1 |

Nota que no estás almacenando los IDs de correo electrónico completos, ¡solo estás configurando un par de bits correspondientes a cada uno!

Para probar si un elemento no está en el filtro Bloom, realizarías los siguientes pasos:

1. Hashea el elemento utilizando las mismas funciones hash.
2. Una vez que tengas los valores hash, verifica si los bits en esas posiciones en el filtro Bloom están configurados. Si alguno de los bits no está configurado, el elemento no está presente en el filtro Bloom.

En nuestro caso, si intentamos insertar "max@gmail.com", los valores hash serían 0 y 9 respectivamente. Dado que el bit en el índice 0 no está configurado, podemos estar seguros de que no existe en nuestra base de datos y podemos proceder con seguridad a crear el nuevo ID de correo electrónico.

### Falsos Positivos en Filtros Bloom

Intentemos insertar "jam@gmail.com". Los valores hash correspondientes serían 6 y 9, respectivamente. Ambos bits ya están configurados. Sin embargo, el ID de correo electrónico no está en nuestro conjunto de datos. Este es el caso en el que te encontrarías con un falso positivo y tendrías que realizar una búsqueda más amplia en caché o en la base de datos. Sin embargo, las búsquedas son mucho más rápidas para los ítems que no existen. Para los ítems donde todos los bits están configurados, es "probable" que existan en el conjunto de datos. Necesitas realizar una operación más amplia para afirmar eso.

Por último, los filtros Bloom no permiten eliminaciones. Una vez en el filtro Bloom, siempre en el filtro Bloom. Sin embargo, hay implementaciones que pueden sortear esta limitación.

### Dónde usarlos en una Entrevista de Diseño de Sistemas

Ahora que entendemos cómo funcionan los filtros Bloom, veamos algunos ejemplos donde son relevantes en entrevistas técnicas. Es muy poco probable que esto surja como una pregunta de entrevista de codificación. Pero si te interesa cómo se codifican, los Filtros Bloom están disponibles en el Framework de Colecciones Guava en Java. Implementaciones similares se pueden encontrar para otros lenguajes. Donde son útiles es en entrevistas de diseño de sistemas. Discutimos un ejemplo anteriormente. Sin embargo, si estás usando un filtro Bloom para buscar si una dirección de correo electrónico ya existe, típicamente tendrías que seguir con una verificación más precisa, como una búsqueda en la base de datos, si el filtro Bloom sugiere que la dirección de correo electrónico podría existir.

Para la entrevista de diseño de sistemas abstractos, aquí hay algunos buenos ejemplos donde los filtros Bloom son útiles:

1. **Servicios de Acortamiento de URL**: En el contexto de diseñar un servicio de acortamiento de URL como TinyURL, los entrevistadores podrían preguntar cómo verificar eficientemente la unicidad de las URL cortas generadas. Los filtros Bloom pueden ayudar a determinar rápidamente si una URL corta dada ya está en uso para evitar colisiones y garantizar la unicidad.

2. **Diseño de un Rastreador Web**: Al discutir el diseño de un rastreador web que indexa páginas web, los entrevistadores podrían preguntar sobre estrategias para evitar volver a visitar URLs ya rastreadas. Los filtros Bloom pueden emplearse para almacenar de manera eficiente un conjunto de URLs visitadas, permitiendo al rastreador determinar rápidamente si una URL ha sido visitada antes o no.

3. **Construcción de una Caché Distribuida**: En el contexto de diseñar un sistema de caché distribuido, los entrevistadores podrían explorar cómo determinar rápidamente si un ítem solicitado está presente en la caché a través de múltiples nodos de caché. Los filtros Bloom pueden ayudar a reducir el número de búsquedas en la caché y la sobrecarga de comunicación en la red al filtrar las solicitudes de ítems que no están en la caché.

4. **Redes de Entrega de Contenidos (CDN)**: Las preguntas de entrevista relacionadas con el diseño de una red de entrega de contenido (CDN) podrían involucrar discusiones sobre cómo almacenar y recuperar contenido de manera eficiente en ubicaciones periféricas. Los filtros Bloom pueden ayudar a los nodos CDN a determinar rápidamente si el contenido solicitado está disponible en su caché, reduciendo así la latencia y mejorando el rendimiento general.

Más allá de los casos de uso populares mencionados, los filtros Bloom son útiles al buscar duplicados en grandes conjuntos de datos, como en el filtrado de spam, la detección de duplicados en publicaciones de redes sociales y anuncios, etc. Productos como Google Chrome, la CDN de Akamai y Microsoft Bing utilizan filtros Bloom. ¡Medium usa filtros Bloom para filtrar recomendaciones duplicadas para sus usuarios!